In [1]:
# 1. Importar librerías
import pandas as pd
import numpy as np
import os
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import joblib

In [2]:
# 2. Cargar dataset desde ruta local
ruta = r"C:\Users\52358965\Proyecto\data\raw\Base.xlsx"
df = pd.read_excel(ruta)

print("✅ Dataset cargado correctamente.")
print("Shape del dataset:", df.shape)
print(df.head())

✅ Dataset cargado correctamente.
Shape del dataset: (24518, 7)
    Año      Sucursal  Mes  Semana    Producto  Valores  Unidades
0  2024  Sucursal 001    1       1  PRODUCTO 1      249         1
1  2024  Sucursal 001    1       1  PRODUCTO 2      378         1
2  2024  Sucursal 001    1       1  PRODUCTO 6      803         2
3  2024  Sucursal 001    1       1  PRODUCTO 7     2857         5
4  2024  Sucursal 001    1       1  PRODUCTO 8     1715         3


In [3]:
# 3. Limpieza y preparación de la variable objetivo

# Limpiar formato monetario en columna 'Valores'
if 'Valores' in df.columns:
    df['Valores'] = df['Valores'].replace('[Q,]', '', regex=True).astype(float)
    target_col = 'Valores'
else:
    raise ValueError("❌ No se encontró la columna 'Valores' en el dataset.")

In [4]:
# 4. variable objetivo y variables predictoras

X = df.drop(columns=[target_col])
y = df[target_col]

In [5]:
# 5. se separa variables numéricas y categóricas

num_features = X.select_dtypes(include=['int64', 'float64']).columns
cat_features = X.select_dtypes(include=['object', 'category']).columns

print("\nVariables numéricas:", list(num_features))
print("Variables categóricas:", list(cat_features))


Variables numéricas: ['Año', 'Mes', 'Semana', 'Unidades']
Variables categóricas: ['Sucursal', 'Producto']


In [6]:
# 6. se define pipelines individuales
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [7]:
# 7. se combina pipelines en preprocesador general
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_pipeline, num_features),
    ('cat', categorical_pipeline, cat_features)
])

In [8]:
# 8. Crear pipeline completo (preprocesamiento + modelo)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

In [9]:
# 9. División de datos y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_pipeline.fit(X_train, y_train)

# Evaluación básica
r2_train = model_pipeline.score(X_train, y_train)
r2_test = model_pipeline.score(X_test, y_test)

print(f"\nR² (entrenamiento): {r2_train:.4f}")
print(f"R² (prueba): {r2_test:.4f}")


R² (entrenamiento): 0.6097
R² (prueba): 0.6161


In [10]:
# 10. Guardar pipeline entrenado

output_path = os.path.join(os.getcwd(), "feature_pipeline.pkl")
joblib.dump(model_pipeline, output_path)

print(f"\n✅ Pipeline guardado exitosamente en: {output_path}")


✅ Pipeline guardado exitosamente en: C:\Users\52358965\Proyecto\notebooks\feature_pipeline.pkl


In [12]:
# 11. Ej. de inferencia

sample = X_test.sample(1, random_state=1)
pred = model_pipeline.predict(sample)
print("\nEj. predicción:")
print(pred)


Ej. predicción:
[254.15309418]
